In [1]:
import SimpleITK as sitk
from radiomics import featureextractor
import nibabel as nib
import numpy as np
from PIL import Image
from pathlib import Path
import os
import matplotlib.pyplot as plt
import csv
import pandas as pd

In [31]:
# Paths relativos de las imágenes y las máscaras
dir_datos = Path("/Users/celiaalme/Desktop/TFM/Practica_Celia/Estudios")
dir_datosROIs = Path("/Users/celiaalme/Desktop/TFM/Practica_Celia/ROIS")

# Creamos el extractor de características
extractor = featureextractor.RadiomicsFeatureExtractor()

# Función para comprobar que cogeremos la máscara que se corresponda con la imagen 
# (comprobación del nombre de los directorios)
def verificar_codigo(dir_img, dir_ROIs):
    codigo = dir_ROIs[:7]
    if codigo in dir_img:
        return True
    else:
        return False  

# Iteramos para acceder a cada carpeta de los estudios

# dir: control 10 semanas, control 20 semanas, HFD 10 semanas, HFD 20 semanas
for dir in os.listdir(dir_datos):
    if not dir.startswith('.'):
        # Creamos una carpeta de CSVs en cada estudio
        dir_name = 'CSV_N4ITK'
        target_dir_name = dir_datos.joinpath(dir).joinpath(dir_name)
        if not os.path.exists(target_dir_name):
            try:
                os.makedirs(target_dir_name)
            except FileExistsError:
                pass
       
        # Creamos dos carpetas por cada carpeta CSV: machos, hembras
        subDir_name_M = 'Machos'
        subDir_name_H = 'Hembras'
        target_subDir_name_M = target_dir_name.joinpath(subDir_name_M)
        target_subDir_name_H = target_dir_name.joinpath(subDir_name_H)
        if not os.path.exists(target_subDir_name_M) and not os.path.exists(target_subDir_name_H):
            try:
                os.makedirs(target_subDir_name_H)
                os.makedirs(target_subDir_name_M)
            except FileExistsError:
                pass
                
        
        # subDir: hembras, machos
        for subDir in os.listdir(dir_datos.joinpath(dir)):
            if not subDir.startswith('.'):
                
                # resoDir: resomapper_output
                for resoDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir)):
                    if not resoDir.startswith('.'):
                        
                    # sourcedataDir: derivatives, sourcedata
                        for sourcedataDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir)):
                            if not sourcedataDir.startswith('.') and sourcedataDir == "sourcedata":
                                
                                # studyDir: estudio
                                for studyDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir)):
                                    if not studyDir.startswith('.'):
                                        for imgfile in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir).joinpath(studyDir)):
                                            if not imgfile.startswith('.') and imgfile.endswith('.nii.gz'):
                                                # Cargamos la imagen y las labels (no hago bucle porque hay distintos tipos de corte); cambiamos el nombre del archivo 
                                                image = sitk.ReadImage(str(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir).joinpath(studyDir).joinpath(imgfile)))
                                                # Seleccionar el índice del slice deseado (el corte 3 es el que mejor se ve)
                                                slice_index = 2
                                                # Obtenemos el tamaño de la imagen
                                                size = list(image.GetSize())
                                                size[2] = 1  # cambiamos el tamaño de la dimensión Z a 1 (para extraer un solo corte)
                                                # Establecer el índice de inicio del corte
                                                index = [0, 0, slice_index]
                                                # Usar el filtro de extracción
                                                ex_slicer = sitk.ExtractImageFilter()
                                                ex_slicer.SetSize(size)
                                                ex_slicer.SetIndex(index)
                                                slice = ex_slicer.Execute(image)  
                                                #print("Espaciado de slice:", slice.GetSpacing())
                                                
                                                # Ajustar el espaciado de la imagen 2D
                                                slice = sitk.Cast(slice, sitk.sitkFloat32)
                                                
                                                
                                                # orig_origin = slice.GetOrigin()
                                                # orig_spacing = slice.GetSpacing()
                                                # orig_direction = slice.GetDirection()

                                                # slice.SetOrigin([0.0, 0.0, 0.0])
                                                # slice.SetSpacing([1.0, 1.0, 1.0])
                                                # slice.SetDirection([1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0])  # Identity 3x3

                                                # print("Espaciado post-reset:", slice.GetSpacing())
                                                
                                                
                                                
                                                # spacing = list(slice.GetSpacing())
                                                # if len(spacing) < 3:
                                                #     spacing.append(1.0)  # Añadir un valor válido para la dimensión Z si es necesario
                                                # else:
                                                #     spacing[2] = 1.0  # Establecer un valor válido para la dimensión Z
                                                # slice.SetSpacing(spacing)
                                                
                                                for dir_ROI in os.listdir(dir_datosROIs):
                                                    if not dir_ROI.startswith('.'):
                                                        for subDir_ROI in os.listdir(dir_datosROIs.joinpath(dir_ROI)):
                                                            if not subDir_ROI.startswith('.'):
                                                                for studyDir_ROI in os.listdir(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):
                                                                    if not studyDir_ROI.startswith('.'):
                                                                        if verificar_codigo(imgfile, studyDir_ROI):
                                                                            
                                                                            # Paso clave: llevamos a cabo la normalización de las imágenes:
                                                                            # esto nos sirve para evitar que las diferencias de intensidad entre imágenes afecten a la extracción de features
                                                                            
                                                                            # Primero, realizamos un suavizado gaussiano para reducir el ruido
                                                                            # Es obligatorio pre-N4 según la documentación oficial de SimpleITK
                                                                            # slice_smooth = sitk.DiscreteGaussian(slice_2D, variance=2.0)
                                                                            
                                                                            
                                                                            #slice = sitk.Cast(slice, sitk.sitkFloat32)
                                                                            
                                                                                
                                                                            # Creamos diccionario para guardar las features de cada ROI y pasarlo a CSV
                                                                            results = {}
                                                                            
                                                                            for maskfile in os.listdir(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(studyDir_ROI)):
                                                                                # Seleccionamos las máscaras en archivo NIfTI
                                                                                if maskfile.endswith('.nii'): 
                                                                                    mask_2D = sitk.ReadImage(str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(studyDir_ROI).joinpath(maskfile)))
                                                                                    mask_2D.SetOrigin(slice.GetOrigin())
                                                                                    mask_2D.SetSpacing(slice.GetSpacing())
                                                                                    mask_2D.SetDirection(slice.GetDirection())
                                                                                    mask_2D = sitk.Cast(mask_2D, sitk.sitkUInt8)
                                                                                   
                                                                                   
                                                                                    print("Propiedades de slice:")
                                                                                    print("Tamaño:", slice.GetSize())
                                                                                    print("Espaciado:", slice.GetSpacing())
                                                                                    print("Origen:", slice.GetOrigin())
                                                                                    print("Dirección:", slice.GetDirection())

                                                                                    print("Propiedades de mask_2D:")
                                                                                    print("Tamaño:", mask_2D.GetSize())
                                                                                    print("Espaciado:", mask_2D.GetSpacing())
                                                                                    print("Origen:", mask_2D.GetOrigin())
                                                                                    print("Dirección:", mask_2D.GetDirection())
                                                                                    
                                                                                    # Cargamos una máscara de referencia para el N4ITK
                                                                                    mask_N4 = sitk.BinaryThreshold(mask_2D, lowerThreshold=1, upperThreshold=255, insideValue=1, outsideValue=0)
                                                                                    # mask_N4.SetOrigin([0.0, 0.0])
                                                                                    # if mask_N4.GetDimension() == 2:
                                                                                    #     mask_N4.SetSpacing([1.0, 1.0])
                                                                                    # else:
                                                                                    #     mask_N4.SetSpacing([1.0, 1.0, 1.0])    
                                                                                    # mask_N4.SetDirection([[1.0, 0.0], [0.0, 1.0]])  # 2D identity
                                                                                    
                                                                                    slice_normalized = sitk.N4BiasFieldCorrection(slice, mask_N4)
                                                                                    
                                                                                    
                                                                                    features = extractor.execute(slice_normalized, mask_N4)
                                                                                    roi_name = maskfile.replace('.nii', '').replace('mask_', '')
                                                                                    results[roi_name] = features

                                                                            results_df = pd.DataFrame(results)
                                                                            csv_filename = f"{studyDir_ROI}.csv"
                                                                            if 'HEMBRAS' in str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):                                                             
                                                                                results_df.to_csv(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(target_subDir_name_H).joinpath(csv_filename))
                                                                            if 'MACHOS' in str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):                                                             
                                                                                results_df.to_csv(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(target_subDir_name_M).joinpath(csv_filename))                                                                   
                                                                        

Propiedades de slice:
Tamaño: (256, 256, 1)
Espaciado: (0.08984375, 0.08984375, 1.0)
Origen: (11.5, 1.132911443710328, 11.500000068457084)
Dirección: (-1.0, -2.7947391750662556e-16, 3.454457115765172e-17, -3.4544560799911886e-17, -3.422854146977085e-08, -0.9999999999999996, 2.794739359203854e-16, -0.9999999999999996, 3.422854177870248e-08)
Propiedades de mask_2D:
Tamaño: (256, 256, 1)
Espaciado: (0.08984375, 0.08984375, 1.0)
Origen: (11.5, 1.132911443710328, 11.500000068457084)
Dirección: (-1.0, -2.7947391750662556e-16, 3.454457115765172e-17, -3.4544560799911886e-17, -3.422854146977085e-08, -0.9999999999999996, 2.794739359203854e-16, -0.9999999999999996, 3.422854177870248e-08)


RuntimeError: Exception thrown in SimpleITK N4BiasFieldCorrection: /Users/ec2-user/actions-runner/_work/SimpleITK/SimpleITK/bld/ITK-prefix/include/ITK-5.4/itkImageBase.hxx:79:
ITK ERROR: Image(0x11fa27090): Zero-valued spacing is not supported and may result in undefined behavior.
Refusing to change spacing from [1, 1, 1] to [22.9102, 22.9102, 0]

In [2]:
print(slice_smooth.GetPixelIDTypeAsString())
print(mask_slice.GetPixelIDTypeAsString())

print("Image size:", slice_smooth.GetSize())
print("Mask size:", mask_slice.GetSize())
print("Image spacing:", slice_smooth.GetSpacing())
print("Mask spacing:", mask_slice.GetSpacing())
print("Image origin:", slice_smooth.GetOrigin())
print("Mask origin:", mask_slice.GetOrigin())
print("Image direction:", slice_smooth.GetDirection())
print("Mask direction:", mask_slice.GetDirection())

mask_array = sitk.GetArrayFromImage(mask_slice)
if not mask_array.any():
    print("La máscara está vacía.")
    
mask_slice = sitk.BinaryThreshold(mask_slice, lowerThreshold=1, upperThreshold=255, insideValue=1, outsideValue=0)


NameError: name 'slice_smooth' is not defined